In [1]:
import pandas as pd
from googleapiclient.discovery import build


import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


#               kirian.fink@gmail.com                       kirianspamf@gmail.com                       kirian123f@gmail.com                                namepwnamepw@gmail.com               kiriantrashf@gmail.com   trashsolid123@gmail.com



# Assuming you already have unique_videos DataFrame
europe = ['AL', 'AT', 'BA', 'BE', 'BG', 'CH', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GB', 'GR',
          'HR', 'HU', 'IE', 'IS', 'IT', 'LI', 'LT', 'LU', 'LV', 'MD', 'ME', 'MK', 'MT', 'NL', 'NO', 'PL',
          'PT', 'RO', 'RS', 'RU', 'SE', 'SI', 'SK', 'UA']

file_path = 'kaggle/combined_df.csv'
df = pd.read_csv(file_path)

# Keep only unique rows of video_id and video_tags
unique_videos = df[['video_id','country', 'video_tags']].drop_duplicates()
print("unique_videos_sorted", unique_videos)


unique_videos['is_europe'] = unique_videos['country'].isin(europe)

# Sort the DataFrame, with European countries first
unique_videos_sorted = unique_videos.sort_values(by='is_europe', ascending=False)
print("unique_videos_sorted", unique_videos_sorted)
unique_videos_sorted.drop(columns=['is_europe'])
print("unique_videos_sorted", unique_videos_sorted)
# Save to CSV
unique_videos_sorted.to_csv('kaggle/unique_videos_europe_in_front.csv', index=True)

unique_videos_sorted             video_id country  \
0        cXmYNmQ4BuM      AE   
1        _wfrNbLguWw      AE   
2        dw50aS4sLHk      AE   
3        ad_BwXUNg_A      AE   
4        ShJw2I8cgBE      AE   
...              ...     ...   
2551477  Ne64UG-BKRo      ZW   
2551487  E5XgV-I9vGo      ZW   
2551494  XoiOOiuH8iI      ZW   
2551496  A27v0njMK_Y      ZW   
2551507  6aea6t0Jb_s      ZW   

                                                video_tags  
0        Jennie Kim, JENNIE MV, JENNIE Solo, Ruby, Mant...  
1        sidhu moosewala, sidhu moosewala latest song, ...  
2                                                      NaN  
3                فالكونز, كودنيمز, كود نيمز, دربحة, فالكون  
4        samsung, phone, smartphone, s25 ultra, hands o...  
...                                                    ...  
2551477                                                NaN  
2551487  SABC News, actuality, world news, local news, ...  
2551494  tyla, new artist, getting late, tyla

In [2]:

def get_video_categories(video_ids):
    """
    Fetch the category IDs for a list of video IDs using the YouTube Data API.
    """
    api_keys = ['Key1','Key2']
    api_key_index = 0
    category_data = {}
    youtube = build('youtube', 'v3', developerKey=api_keys[api_key_index])

    # Split video_ids into batches of 50
    for i in range(0, len(video_ids), 50):
        batch_ids = video_ids[i:i + 50]
        print("Batch: ",i,"  / ",len(video_ids), "   call:",i/50)
        try:
            request = youtube.videos().list(
                part='snippet',
                id=','.join(batch_ids)
            )
            response = request.execute()
        except HttpError as e:
            print(f"API error: {e}")
            api_key_index = api_key_index+1

            if api_key_index < len(api_keys):
                youtube = build('youtube', 'v3', developerKey=api_keys[api_key_index])

                request = youtube.videos().list(
                    part='snippet',
                    id=','.join(batch_ids)
                )
                response = request.execute()
            else:
                print("All API keys exhausted. Stopping the process.")
                return category_data  # Return the data collected so far
        except Exception as e:
            print(f"Unexpected error: {e}")
            return category_data  # Return data if an unexpected error occurs

        for item in response.get('items', []):
            video_id = item['id']
            category_id = item['snippet'].get('categoryId')
            category_data[video_id] = category_id

    return category_data

In [3]:
video_ids = unique_videos['video_id'].unique().tolist()

# Step 1: Get category IDs for each video
video_category_ids = get_video_categories(video_ids)


# Step 3: Map category names back to the DataFrame
unique_videos['category_id'] = unique_videos['video_id'].map(video_category_ids)


print(unique_videos)

unique_videos.to_csv('kaggle/unique_videos_with_category.csv', index=True)

Batch:  0   /  203575    call: 0.0
Batch:  50   /  203575    call: 1.0
Batch:  100   /  203575    call: 2.0
Batch:  150   /  203575    call: 3.0
Batch:  200   /  203575    call: 4.0
Batch:  250   /  203575    call: 5.0
Batch:  300   /  203575    call: 6.0
Batch:  350   /  203575    call: 7.0
Batch:  400   /  203575    call: 8.0
Batch:  450   /  203575    call: 9.0
Batch:  500   /  203575    call: 10.0
Batch:  550   /  203575    call: 11.0
Batch:  600   /  203575    call: 12.0
Batch:  650   /  203575    call: 13.0
Batch:  700   /  203575    call: 14.0
Batch:  750   /  203575    call: 15.0
Batch:  800   /  203575    call: 16.0
Batch:  850   /  203575    call: 17.0
Batch:  900   /  203575    call: 18.0
Batch:  950   /  203575    call: 19.0
Batch:  1000   /  203575    call: 20.0
Batch:  1050   /  203575    call: 21.0
Batch:  1100   /  203575    call: 22.0
Batch:  1150   /  203575    call: 23.0
Batch:  1200   /  203575    call: 24.0
Batch:  1250   /  203575    call: 25.0
Batch:  1300   /  2